In [1]:
import yaml
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

In [2]:
with open('configs.yaml', 'r', encoding='utf-8') as stream:
    try:
        configs = yaml.safe_load(stream)
    except Exception as e:
        print("Có lỗi xảy ra khi đọc file configs.yaml")
        print(str(e))
configs

{'auth': {'email': '19497581.tuan@student.iuh.edu.vn', 'pass': 'Tuan1211'},
 'waiting_time': 240,
 'sign_in_url': 'https://id.elsevier.com/as/authorization.oauth2?platSite=SD%2Fscience&scope=openid%20email%20profile%20els_auth_info%20els_idp_info%20els_idp_analytics_attrs%20els_sa_discover%20urn%3Acom%3Aelsevier%3Aidp%3Apolicy%3Aproduct%3Aindv_identity&response_type=code&redirect_uri=https%3A%2F%2Fwww.sciencedirect.com%2Fuser%2Fidentity%2Flanding&authType=SINGLE_SIGN_IN&prompt=login&client_id=SDFE-v3&state=retryCounter%3D0%26csrfToken%3Dcabfefe6-cfd0-4aaa-ab9a-6b4ab254ecb8%26idpPolicy%3Durn%253Acom%253Aelsevier%253Aidp%253Apolicy%253Aproduct%253Aindv_identity%26returnUrl%3D%252Fsearch%253Foffset%253D5900%2526qs%253Dsmart%252520retailing%2526show%253D100%26prompt%3Dlogin%26cid%3Datp-68519aa1-e6ba-4ab5-b45b-fd75c991b759',
 'url': 'https://www.sciencedirect.com/search?qs=smart%20retailing&show=100&offset={}'}

In [3]:
url = configs['url'].format(100)
url

'https://www.sciencedirect.com/search?qs=smart%20retailing&show=100&offset=100'

In [4]:
driver = webdriver.Chrome()

In [5]:
def find_element(css_selector):
    return driver.find_element(By.CSS_SELECTOR, css_selector)

def find_elements(css_selector):
    return driver.find_elements(By.CSS_SELECTOR, css_selector)

def waiting(css_selector, time=5):
    return WebDriverWait(driver, time).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
    )

def scroll_bottom():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [6]:
driver.get(configs['sign_in_url'])

In [7]:
find_element('input#bdd-email').send_keys(configs['auth']['email'])
find_element('button#bdd-elsPrimaryBtn').click()
time.sleep(1)
find_element('input#bdd-password').send_keys(configs['auth']['pass'])
find_element('button#bdd-elsPrimaryBtn').click()

# Get urls

In [8]:
# %%time

# list_urls = []
# for i in range(60):
#     url = configs['url'].format(i * 100)
#     driver.get(url)
#     href_css = 'a.result-list-title-link.u-font-serif.text-s'
#     try:
#         waiting(href_css)
#         list_elements = find_elements(href_css)
#         list_elements = list(map(lambda x: x.get_attribute('href'), list_elements))
#         list_urls.extend(list_elements)
#     except:
#         print("Done")
#         break

# with open('list_urls.json', 'w') as f:
#     json.dump(list_urls, f, indent=4)

In [9]:
with open('list_urls.json', 'r') as f:
    list_urls = json.load(f)
    
len(list_urls)

6000

In [10]:
def get_a_source(url):
    driver.get(url)
    scroll_bottom()
    try:
        title = waiting('span.title-text').text
    except:
        title = ''
    try:
        abstract = waiting('div.abstract.author > div > p').text
    except:
        abstract = ''
    try:
        keywords = find_elements('div.keyword')
        keywords = list(map(lambda x: x.text, keywords))
    except:
        keywords = []
    data = dict(
        title=title,
        abstract=abstract,
        keywords=keywords
    )
    return data

In [11]:
with open('data.json', 'r') as f:
    data = json.load(f)
len(data)

5306

In [12]:
import sys
import os

In [13]:
%%time

last_epoch = len(data)
for idx, url in enumerate(last_epoch, list_urls):
    print(f'Get source {idx + 1}', end='\r')
    datum = get_a_source(url)
    data.append(datum)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=99.0.4844.74)
Stacktrace:
Backtrace:
	Ordinal0 [0x003F69A3+2582947]
	Ordinal0 [0x0038A6D1+2139857]
	Ordinal0 [0x00283960+1063264]
	Ordinal0 [0x00278092+1015954]
	Ordinal0 [0x002788C8+1018056]
	Ordinal0 [0x0027A102+1024258]
	Ordinal0 [0x00273DE9+998889]
	Ordinal0 [0x00284DD0+1068496]
	Ordinal0 [0x002D7B12+1407762]
	Ordinal0 [0x002C8366+1344358]
	Ordinal0 [0x002A5176+1200502]
	Ordinal0 [0x002A6066+1204326]
	GetHandleVerifier [0x0059BE02+1675858]
	GetHandleVerifier [0x0065036C+2414524]
	GetHandleVerifier [0x0048BB01+560977]
	GetHandleVerifier [0x0048A8D3+556323]
	Ordinal0 [0x0039020E+2163214]
	Ordinal0 [0x00395078+2183288]
	Ordinal0 [0x003951C0+2183616]
	Ordinal0 [0x0039EE1C+2223644]
	BaseThreadInitThunk [0x75B36739+25]
	RtlGetFullPathName_UEx [0x77848E7F+1215]
	RtlGetFullPathName_UEx [0x77848E4D+1165]


In [14]:
with open('data.json', 'w') as f:
    json.dump(data, f, indent=4)

In [15]:
import datetime

In [16]:
datetime.datetime.now()

datetime.datetime(2022, 3, 17, 7, 23, 53, 733633)